### <font color = 'gold'> Import packages


In [12]:
# import packages
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd

### <font color = 'gold'> Collect data over one page

In [13]:
# url
url = 'https://www.francefleurs.com/11-fleurs-coupees-do-it-yourself'
# get the code html of the page using the get function requests
res = get(url)
# stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup = bs(res.text,'html.parser')

In [14]:
# get all containers that contains this information of each flower
containers = soup.find_all('li', class_ = 'ajax_block_product col-xs-12 col-sm-6 col-md-4')
# length de containers
len(containers)

60

First, we will start with one container and scrape the data. In that container, we will first try to get the URL of the container and enter inside the container to scrape the informations needed. After we generalize the scraping to the other containers by using a for loop.

In [24]:
# scrape data from the first container
container = containers[0]
# get the url of the container
link = container.find('a')['href']
# get the html code of the container
res_c = get(link)
# store the html code in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup_c = bs(res_c.text, 'html.parser')
# get the details
details = soup_c.find('div', class_ = "product-title-flex").text.strip()

# get the price
price = soup_c.find('span', class_ = "price").text.strip().replace(" €", "").replace(",", ".")

#details_price = details[1]

# get the rating
rating = soup_c.find('span', class_ = "ratingValue").text

# get number of opinions
num_opinions = soup_c.find('a', class_ = "netreviewsProductWidgetNewLink").span.text
# get image link

image_link = soup_c.find('div', class_ = "image-block clearfix hidden-xs hidden-sm").img['src']

# print all data for the first container
print(details,"------",price,"------",rating,"-----",num_opinions,"------",image_link)

Rose branchue bombastic blanche (10 tiges) ------ 27.90 ------ 2.4 ----- 5 ------ https://www.francefleurs.com/17992-large_default/rose-branchue-bombastic-blanche.jpg


In [15]:
# scrape data from all the containers
data = []

for container in containers :
    try:
        
        link = container.find('a')['href']
        # get the html code of the container
        res_c = get(link)
        # store the html code in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
        soup_c = bs(res_c.text, 'html.parser')
        # get the details
        details = soup_c.find('div', class_ = "product-title-flex").text.strip()

        # get the price
        price = soup_c.find('span', class_ = "price").text.strip().replace(" €", "").replace(",", ".")

        #details_price = details[1]

        # get the rating
        rating = soup_c.find('span', class_ = "ratingValue").text

        # get number of opinions
        num_opinions = soup_c.find('a', class_ = "netreviewsProductWidgetNewLink").span.text
        # get image link

        image_link = soup_c.find('div', class_ = "image-block clearfix hidden-xs hidden-sm").img['src']
        
        dic = {'details':details,
           'price' : price,
           'rating' : rating,
            'num_opinions' : num_opinions,
            'image_link' : image_link }
        data.append(dic)
    except:
        pass
    
df = pd.DataFrame(data)

In [27]:
# data from the first page
df.head()

,details,price,rating,num_opinions,image_link
0,Rose branchue bombastic blanche (10 tiges),27.90,2.4,5,https://www.francefleurs.com/17992-large_defau...
1,Brassée deuil blanche,29.90,4.2,5,https://www.francefleurs.com/17748-large_defau...
2,Brassée de fleurs fraîches blanches,16.90,5,2,https://www.francefleurs.com/17742-large_defau...
3,Roses emballées à l'unité,86.90,4.2,61,https://www.francefleurs.com/10602-large_defau...
4,Brassée de fleurs fraîches surprise,39.00,3.7,14,https://www.francefleurs.com/15052-large_defau...


### <font color = 'gold'> Collect data over multiple pages

In [11]:
# create a empty dataframe df
df = pd.DataFrame()

for p in range(1,5) :
    
        # url
    url = f'https://www.francefleurs.com/11-fleurs-coupees-do-it-yourself?p={p}'
    # get the code html of the page using the get function requests
    res = get(url)
    # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
    soup = bs(res.text,'html.parser')
    # get all containers that contains this information of each flower
    containers = soup.find_all('li', class_ = 'ajax_block_product col-xs-12 col-sm-6 col-md-4')   

    # scrape data from all the containers
    data = []

    for container in containers :
        try:

            link = container.find('a')['href']
            # get the html code of the container
            res_c = get(link)
            # store the html code in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
            soup_c = bs(res_c.text, 'html.parser')
            # get the details
            details = soup_c.find('div', class_ = "product-title-flex").text.strip()

            # get the price
            price = soup_c.find('span', class_ = "price").text.strip().replace(" €", "").replace(",", ".")

            #details_price = details[1]

            # get the rating
            rating = soup_c.find('span', class_ = "ratingValue").text

            # get number of opinions
            num_opinions = soup_c.find('a', class_ = "netreviewsProductWidgetNewLink").span.text
            # get image link

            image_link = soup_c.find('div', class_ = "image-block clearfix hidden-xs hidden-sm").img['src']

            dic = {'details':details,
               'price' : price,
               'rating' : rating,
                'num_opinions' : num_opinions,
                'image_link' : image_link }
            data.append(dic)
        except:
            pass

        DF = pd.DataFrame(data)

    df = pd.concat([df,DF], axis = 0).reset_index(drop =True)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   details       175 non-null    object
 1   price         175 non-null    object
 2   rating        175 non-null    object
 3   num_opinions  175 non-null    object
 4   image_link    175 non-null    object
dtypes: object(5)
memory usage: 7.0+ KB


In [8]:
df['rating'].unique()

array(['2.4', '4.2', '5', '3.7', '2.8', '4.9', '3.4', '4.4', '3.9', '4.8',
       '3.8', '4.1', '4', '4.5', '4.6', '3.6', '4.3', '4.7', '3.5', '3.3',
       '3', '2', '3.1', '2.7', '2.5', '2.9', '2.3'], dtype=object)

### <font color = 'gold'> Assignment

In [ ]:
# url
url = "https://www.francefleurs.com/231-les-fleurs-de-mon-mariage"
# scrape over all pages the following informations : details, price, rating, number of opinions, image link
# Clean the data